In [1]:
from lxml.html import fromstring
import requests
import pandas as pd
from IPython.display import Markdown

#INICIA FIREBASE
import firebase_admin
from firebase_admin import firestore

cred_obj = firebase_admin.credentials.Certificate('./apprenova-c3758-firebase-adminsdk-99epx-89d70a8235.json')
default_app = firebase_admin.initialize_app(cred_obj, {
    'databaseURL': '/'
})
db = firestore.client()
ref = db.collection(u'2006').document('KdHuXcU3T8cecJJ8CZk3')


#LISTA DE CIDADES, NUMERO IDENTIFICADOR E URL DE ACESSO.
def consulta():
    '''Consulta os resultados de todas as cidades'''
    url = 'https://resultados.tre-rs.jus.br/eleicoes/2006/1turno/'
    req = requests.get(url)
    root = fromstring(req.text)
    ul = root.xpath('.//ul[@id="main_ul"]')[0]
    cities = []
    for li in ul.xpath('.//li')[0:]:
        cities.append({
            'name': li.xpath('.//a')[0].text_content(),
            'href': li.xpath('.//a')[0].attrib['href'],
            'url': url + li.xpath('.//a')[0].attrib['href']
        })

    #display(pd.DataFrame(cities)) #IMPRIMIR EM FORMATO DE TABELA AS INFORMAÇÕES DAS CIDADES

    #LISTA DE ELEITOS DO ACESSO AO RS 
    url = cities[0]["url"]
    req = requests.get(url)
    root = fromstring(req.text)
    tables = root.xpath('.//table[@summary="Votação dos candidatos na UF"]')
    h2_s = root.xpath('.//h2')
    data = []
    eleitos_class = ["lr_even eleito ", "eleito "]
    id = cities[0]["href"]
    city = cities[0]["name"]
    for index,table in enumerate(tables):
        data.append({
            'id': id,
            'city': city,
            'year': "2006",
            'turno': "PRIMEIRO TURNO",
            'title': h2_s[index].text_content(),
            'values': []
        })
        for classe in eleitos_class:
            for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                data[-1]['values'].append({ 
                'name': tr.xpath('.//a')[0].text_content(),
                'partido': tr.xpath('.//td')[3].text_content(),
                'votação': tr.xpath('.//td')[4].text_content()
            })
            


    #ELEITOS SEGUNDO TURNO
    url2 = 'https://resultados.tre-rs.jus.br/eleicoes/2006/2turno/RS.html'
    req = requests.get(url2)
    root = fromstring(req.text)
    tables = root.xpath('.//table[@summary="Votação dos candidatos na UF"]')
    h2_s = root.xpath('.//h2')
    eleitos_class = ["lr_even eleito ", "eleito "]
    for index,table in enumerate(tables):
        data.append({
            'id': id,
            'city': city,
            'year': "2006",
            'turno': "SEGUNDO TURNO",
            'title': h2_s[index].text_content(),
            'values': []
        })
        for classe in eleitos_class:
            for tr in table.xpath(f'.//tr[@class="{classe}"]'):
                data[-1]['values'].append({ 
                'name': tr.xpath('.//a')[0].text_content(),
                'partido': tr.xpath('.//td')[3].text_content(),
                'votação': tr.xpath('.//td')[4].text_content()
            })

    # IMPRIMIR AS INFORMAÇÕES EM FORMATO DE TABELA
    # for d in data:
    #     display(Markdown(f"# PRIMEIRO TURNO"))
    #     display(Markdown(f"## {d['title']}"))
    #     display(pd.DataFrame(d['values']))


    for d in data:
        if d['title'] == 'Governador' and d['turno'] == 'SEGUNDO TURNO':
            for i in data:
                if i['title'] == 'Governador' and i['turno'] == 'PRIMEIRO TURNO':
                    data.remove(i)
    for d in data:
        if (d['title'] == 'Presidente'):
            data.remove(d)
        
    # REGISTRA OD DADOS NO FIREBASE, MERGE TRUE NÃO SUBSTITUI O CONTEÚDO QUE JÁ ESTÁ NA COLEÇÃO
    for d in data:
        if d['title'] == 'Governador':
            ref.set({
                'Governador': d
            }, merge=True)
        elif d['title'] == 'Senador':
            ref.set({
                'Senador': d
            }, merge=True)
        elif d['title'] == 'Deputado Federal':
            ref.set({
                'Deputado Federal': d
            }, merge=True)
        elif d['title'] == 'Deputado Estadual':
            ref.set({
                'Deputado Estadual': d
            }, merge=True)
    
    return data

consulta()


[{'id': 'RS.html',
  'city': 'Rio Grande do Sul',
  'year': '2006',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Senador',
  'values': [{'name': 'PEDRO JORGE SIMON',
    'partido': 'PMDB',
    'votação': '1862560'}]},
 {'id': 'RS.html',
  'city': 'Rio Grande do Sul',
  'year': '2006',
  'turno': 'PRIMEIRO TURNO',
  'title': 'Deputado Federal',
  'values': [{'name': 'LUIS CARLOS HEINZE',
    'partido': 'PP',
    'votação': '205734'},
   {'name': 'LUCIANA KREBS GENRO', 'partido': 'PSOL', 'votação': '185071'},
   {'name': 'VILSON LUIZ COVATTI', 'partido': 'PP', 'votação': '168320'},
   {'name': 'ELISEU LEMOS PADILHA', 'partido': 'PMDB', 'votação': '140494'},
   {'name': 'JORGE ALBERTO PORTANOVA MENDES RIBEIRO FILHO',
    'partido': 'PMDB',
    'votação': '115245'},
   {'name': 'MARIA DO ROS�RIO NUNES', 'partido': 'PT', 'votação': '110081'},
   {'name': 'TARCISIO JO�O ZIMERMMANN', 'partido': 'PT', 'votação': '106659'},
   {'name': 'OSMAR GASPARINI TERRA', 'partido': 'PMDB', 'votação': '101695'